In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('src')

In [2]:
import json
from src.utils import recursive_objectify, recursive_dictify

params = recursive_objectify(json.load(open('default_params.json')))

model_params = params['model_params']
train_params = params['train_params']
eval_params = params['eval_params']
optimizer_params = params['optimizer_params']
data_params = params['data_params']

In [3]:
from src.model import get_model

model_name = "ice_slider"
checkpoint_path = f"/home/admin/PPGS/results/{model_name}/{model_name}.tor"

model = get_model(model_params=model_params, train_params=train_params, eval_params=eval_params,
                  optimizer_params=optimizer_params, data_params=data_params, device='cuda')

model.load(checkpoint_path)

In [5]:
from functools import partial
from src.planners import MCBFSPlanner, FullPlanner, BaselinePlanner
from src.planners.algos import mc_bfs, selective_mc_bfs, flat_mc_bfs

mcbfs = partial(MCBFSPlanner, search_algo=mc_bfs)
full = partial(FullPlanner, search_algo=selective_mc_bfs)
base = partial(BaselinePlanner, search_algo=None)

In [5]:
from src.evaluation import evaluate_with_planning

planners = {
    'base': base, 
    # 'mcbfs': mcbfs,
    # 'full': full
}

eval_params = recursive_dictify(params.eval_params)
eval_params['seeds'] = [2000]
eval_params['num_levels'] = 30
eval_params['margin'] = 0.05
eval_params['eval_at'] = [1, 3, 4]
# Evaluate with each planner
results = {}
for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")
    metrics = evaluate_with_planning(
        planner=planner,
        model_dict=model.model_dict,
        eval_params=recursive_objectify(eval_params),
        data_params=data_params,
        device='cuda'
    )
    results[planner_name] = metrics
    
    # Print metrics for this planner
    print(f"Results for {planner_name}:")
    for k, v in metrics.items():
        print(f"  {k}: {v}")
    print()

Evaluating with planner: base
Results for base:
  plan_acc_1_2000: 0.5666666664777777
  plan_acc_3_2000: 0.4333333331888889
  plan_acc_4_2000: 0.4999999998333333
  lvl_solved_ratio_2000: 0.23333333325555555
  speedup_2000: 1.023832070705455



In [6]:
planners = {
    # 'base': base, 
    'mcbfs': mcbfs,
    'full': full
}

for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")

    eval_params = recursive_dictify(params.eval_params)
    eval_params['seeds'] = [2000]
    eval_params['num_levels'] = 30
    eval_params['eval_at'] = [1, 3, 4]
    # Evaluate with each planner
    results = {}
    for m in [0.01, 0.05, 0.1, 0.2, 0.5, 1]:
        eval_params['planner_params']['margin'] = m
        metrics = evaluate_with_planning(
            planner=planner,
            model_dict=model.model_dict,
            eval_params=recursive_objectify(eval_params),
            data_params=data_params,
            device='cuda'
        )
        results[planner_name] = metrics
        print(f"margin={m}, pct_solved={metrics['lvl_solved_ratio_2000']}")

Evaluating with planner: mcbfs
margin=0.01, pct_solved=0.5666666664777777
margin=0.05, pct_solved=0.7999999997333334
margin=0.1, pct_solved=0.7666666664111111
margin=0.2, pct_solved=0.7333333330888888
margin=0.5, pct_solved=0.4999999998333333
margin=1, pct_solved=0.06666666664444444
Evaluating with planner: full


KeyboardInterrupt: 

In [8]:
planners = {
    # 'base': base, 
    # 'mcbfs': mcbfs,
    'full': full
}

for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")

    eval_params = recursive_dictify(params.eval_params)
    eval_params['seeds'] = [2000]
    eval_params['num_levels'] = 30
    eval_params['eval_at'] = [1, 3, 4]
    # Evaluate with each planner
    results = {}
    for m in [0.05, 0.1, 0.2, 0.5, 1]:
        eval_params['planner_params']['margin'] = m
        metrics = evaluate_with_planning(
            planner=planner,
            model_dict=model.model_dict,
            eval_params=recursive_objectify(eval_params),
            data_params=data_params,
            device='cuda'
        )
        results[planner_name] = metrics
        print(f"margin={m}, pct_solved={metrics['lvl_solved_ratio_2000']}")

Evaluating with planner: full
margin=0.05, pct_solved=0.9333333330222222
margin=0.1, pct_solved=0.9666666663444444
margin=0.2, pct_solved=0.9333333330222222
margin=0.5, pct_solved=0.7666666664111111
margin=1, pct_solved=0.3999999998666667


In [9]:
planners = {
    # 'base': base, 
    # 'mcbfs': mcbfs,
    'full': full
}

for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")

    eval_params = recursive_dictify(params.eval_params)
    eval_params['seeds'] = [2000]
    eval_params['num_levels'] = 30
    eval_params['eval_at'] = [1, 3, 4]
    # Evaluate with each planner
    results = {}
    for m in [0.01]:
        eval_params['planner_params']['margin'] = m
        metrics = evaluate_with_planning(
            planner=planner,
            model_dict=model.model_dict,
            eval_params=recursive_objectify(eval_params),
            data_params=data_params,
            device='cuda'
        )
        results[planner_name] = metrics
        print(f"margin={m}, pct_solved={metrics['lvl_solved_ratio_2000']}")

Evaluating with planner: full
margin=0.01, pct_solved=0.9333333330222222


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from src.model import get_model

model_name = "digit_jump"
checkpoint_path = f"/home/admin/PPGS/results/{model_name}/{model_name}.tor"

model = get_model(model_params=model_params, train_params=train_params, eval_params=eval_params,
                  optimizer_params=optimizer_params, data_params=data_params, device='cuda')

model.load(checkpoint_path)

In [6]:
from src.evaluation import evaluate_with_planning

planners = {
    'base': base, 
    # 'mcbfs': mcbfs,
    # 'full': full
}

data_params = recursive_dictify(data_params)
data_params['env_name'] = model_name
data_params = recursive_objectify(data_params)
eval_params = recursive_dictify(params.eval_params)
eval_params['seeds'] = [2000]
eval_params['num_levels'] = 30
eval_params['margin'] = 0.05
eval_params['eval_at'] = [1, 3, 4]
# Evaluate with each planner
results = {}
for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")
    metrics = evaluate_with_planning(
        planner=planner,
        model_dict=model.model_dict,
        eval_params=recursive_objectify(eval_params),
        data_params=data_params,
        device='cuda'
    )
    results[planner_name] = metrics
    
    # Print metrics for this planner
    print(f"Results for {planner_name}:")
    for k, v in metrics.items():
        print(f"  {k}: {v}")
    print()

Evaluating with planner: base
Results for base:
  plan_acc_1_2000: 0.5999999998
  plan_acc_3_2000: 0.4666666665111111
  plan_acc_4_2000: 0.26666666657777777
  lvl_solved_ratio_2000: 0.13333333328888888
  speedup_2000: 1.0274878729956969



In [8]:
planners = {
    # 'base': base, 
    # 'mcbfs': mcbfs,
    'full': full
}

for planner_name, planner in planners.items():
    print(f"Evaluating with planner: {planner_name}")

    eval_params = recursive_dictify(params.eval_params)
    eval_params['seeds'] = [2000]
    eval_params['num_levels'] = 30
    eval_params['eval_at'] = [1, 3, 4]
    # Evaluate with each planner
    results = {}
    for m in [0.05, 0.1, 0.2, 0.5, 1]:
        eval_params['planner_params']['margin'] = m
        metrics = evaluate_with_planning(
            planner=planner,
            model_dict=model.model_dict,
            eval_params=recursive_objectify(eval_params),
            data_params=data_params,
            device='cuda'
        )
        results[planner_name] = metrics
        print(f"margin={m}, pct_solved={metrics['lvl_solved_ratio_2000']}")

Evaluating with planner: full
margin=0.05, pct_solved=0.9666666663444444
margin=0.1, pct_solved=0.9999999996666666
margin=0.2, pct_solved=0.9999999996666666
margin=0.5, pct_solved=0.8666666663777778
margin=1, pct_solved=0.1666666666111111
